<a href="https://colab.research.google.com/github/darshvaghasia12/Awesome-Web-Art/blob/master/Handwritten_Digit_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras
!pip install pillow

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as k

# The Data Split
(X_train,Y_train),(X_test,Y_test)=mnist.load_data()

print(X_train.shape,Y_train.shape)


11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) (60000,)


In [ ]:
#Preprocess the Data
#The image data cannot be fed directly into the model so we need to perform some operations and process the data to make it ready for our neural network. 
#The dimension of the training data is (60000,28,28). The CNN model will require one more dimension so we reshape the matrix to shape (60000,28,28,1).
X_train=X_train.reshape(X_train.shape[0],28,28,1)
X_test=X_test.reshape(X_test.shape[0],28,28,1)
input_shape=(28,28,1)

#Converting class vectors to binary class matrices 
Y_train=keras.utils.to_categorical(Y_train,num_classes=None)
Y_test=keras.utils.to_categorical(Y_test,num_classes=None)
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
#Normalizing the vectors since it is Image and the Maximum Value of the pixel is 255 So we will Simply divide it with 255
X_train/=255
X_test/=255

print('X_train.shape:',X_train.shape)
print(X_train.shape[0],'train samples')
print(X_test.shape[0],'test samples')

X_train.shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
#Create the Model
batch_size=128
epochs=10
num_classes=10

model=Sequential()
model.add(Conv2D(32,kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64,(3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [ ]:



hist=model.fit(X_train,Y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test,Y_test))
print('The Model has successfully trained')
model.save('mnist.h5')
print("Saving the model as mnist.h5")

Epoch 1/10
469/469 [==============================] - 163s 347ms/step - loss: 2.2899 - accuracy: 0.1310 - val_loss: 2.2479 - val_accuracy: 0.3009
Epoch 2/10
469/469 [==============================] - 159s 340ms/step - loss: 2.2434 - accuracy: 0.2435 - val_loss: 2.1858 - val_accuracy: 0.4327
Epoch 3/10
469/469 [==============================] - 158s 336ms/step - loss: 2.1807 - accuracy: 0.3493 - val_loss: 2.0992 - val_accuracy: 0.5397
Epoch 4/10
469/469 [==============================] - 162s 346ms/step - loss: 2.0941 - accuracy: 0.4426 - val_loss: 1.9751 - val_accuracy: 0.6176
Epoch 5/10
469/469 [==============================] - 162s 345ms/step - loss: 1.9722 - accuracy: 0.5156 - val_loss: 1.7992 - val_accuracy: 0.6822
Epoch 6/10
469/469 [==============================] - 162s 346ms/step - loss: 1.7932 - accuracy: 0.5880 - val_loss: 1.5682 - val_accuracy: 0.7398
Epoch 7/10
469/469 [==============================] - 163s 347ms/step - loss: 1.5843 - accuracy: 0.6357 - val_loss: 1.3111 -

In [ ]:
!pip install win32gui

  Using cached https://files.pythonhosted.org/packages/b8/75/7bed82934e51903f9d48b26b3996161bb2dce1731607b4bb7fd26003ed3e/win32gui-221.5.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 /usr/local/lib/python3.6/dist-packages/pip/_vendor/pep517/_in_process.py get_requires_for_build_wheel /tmp/tmpjkhot5_q Check the logs for full command output.


In [ ]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

model=load_model("mnist.h5")

def predict_digit(img):
  #resize image to 28x28 pixels
  img=img.resize((28,28))
  #convert rgb to grayscale
  img=img.convert('L')
  img=np.array(img)
  #reshaping to support our model and normalizing
  img=img.reshape(1,28,28,1)
  img/=255.0
  #predicting the class
  res=model.predict([img])[0]
  return np.argmax(res), max(res)

  class App(tk.Tk):
    def __init__(self):
      tk.Tk.__init__(self)
      self.x=self.y=0

      #creating elements
      self.canvas=tk.canvas(self,width=300,height=300,bg="white",cursor="cross")
      self.label=tk.label(self,text="Thinking..",font=("Helvetica",48))
      self.classify_btn=tk.Button(self,text="recognise",command=self.classify_handwriting)
      self.button_clear=tk.Button(self,text="Clear",command=self.clear_all)

      #Grid Structure 
      self.canvas.grid(row=0,column=0,pady=2,sticky=W,)
      self.label.grid(row=0,column=1,pady=2,padx=2)
      self.classify_btn(row=1,column=1,pady=2,padx=2)
      self.button_clear(row=1,column=0,pady=2)

      #self.canvas.bind("<Motion>",self.start_pos)
      self.canvas.bind("<B1-Motion>",self.draw_lines)

    def clear_all(self):
       self.canvas.delete("all")

    def classify_handwriting(self):
      HWND=self.canvas.winfo_id()#to get the handle of the canvas
      rec=win32gui.GetWindowRec(HWND)#get the coordinates of the canvas
      im=ImageGrab.grab(rec)

      digit,acc=predict_digit(im)
      self.label.configure(text=str(digit)+', '+str(int(acc*100))+'%')

    def draw_lines(self,event):
      self.x=event.x
      self.y=event.y
      r=8
      self.canvas.create_oval(self.x-r,self.y-r,self.x+r,self.y+r,fill='black')

    app=App()
    mainloop()


ModuleNotFoundError: ignored